# Modding Minecraft
## Modding Python

## Installation

Go to http://www.wiley.com/WileyCDA/Section/id-823690.html and download the **Starter Kit** for your platform (I used the Mac Starter Kit to run on Linux).  The **readme.txt** included will walk you through getting up and running.  It looks like **Bukkit** is now a defunct project, so I used **CanaryMod**.

The major components you will need:

* CanaryMod (http://canarymod.net/releases)
* RaspberryJuice (https://github.com/martinohanlon/canaryraspberryjuice)
* Minecraft (https://minecraft.net)

Checkout the code from this presentation (https://github.com/tobynance/okcpython.git)
> cd okcpython/presentation/2015_07_08_modding_minecraft/code

### Make Sure To...
Open the eula.txt file in the Canarymod folder and change:
> eula=false

to:

> eula=true

Now Run CanaryMod:
> java -Xmx1024M -jar Canarymod.jar

And execute the server command:
> playermod group add `<playername>` admins

## Running

After you have followed the installation steps in the **readme.txt** of the Starter Kit:

* Start Minecraft
* Start CanaryMod
> java -Xmx1024M -jar Canarymod.jar

* Connect to your local CanaryMod server
  + In Minecraft, choose **Multiplayer**, click Add server
  + For the **Server Name** I used *`Canary`*
  + For the **Server Address** put *`localhost`*
  + Click **Done**
  
![Add Server](minecraft_adding_server.png)
  
You should now have the Canary server showing in the Server List

![Server List](minecraft_server_list.png)

In [5]:
from code.mcpi import block
from code.mcpi import minecraft

########################################################################
def floor_of_dirt():
    world = minecraft.Minecraft.create()
    player = world.player

    pos = player.getTilePos()
    width = 10
    world.setBlocks(pos.x-width, pos.y-1, pos.z-width, pos.x+width, pos.y-1, pos.z+width, block.DIRT.id)

########################################################################
floor_of_dirt()